In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
import datetime
torch.cuda.is_available()
torch.cuda.current_device()

In [ ]:
data_path = '/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train = True, download = True, transform =  transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))

In [ ]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size = 64, shuffle = True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size = 64, shuffle = False)
correct = 0
total = 0

In [ ]:
from torch.nn.modules.activation import Tanh
from torch.nn.modules.linear import Linear
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.conv import Conv2d
model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size= 3, padding= 1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 8, kernel_size= 3, padding= 1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(512, 32),
            nn.Tanh(),
            nn.Linear(32,10)
)
model.to('cuda:0')

In [ ]:
learning_rate = 3e-3
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()
loss_fn.to('cuda:0')
n_epochs = 300

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
  for epoch in range(1, n_epochs +1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      outputs = model(imgs.to('cuda:0'))
      loss = loss_fn(outputs.to('cuda:0'), labels.to('cuda:0'))
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()
      if epoch == 1 or epoch % 10 == 0:
        print('Epoch {}, Training Loss {}'.format( epoch, loss_train / len(train_loader)))

In [ ]:
training_loop(
    n_epochs = n_epochs,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

In [ ]:
def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0
    with torch.no_grad():
      for imgs, labels in loader:
        outputs = model(imgs.to('cuda:0'))
        _, predicted = torch.max(outputs.to('cuda:0'), dim = 1)
        total += labels.shape[0]
        correct += int((predicted.to('cuda:0') == labels.to('cuda:0')).sum())
    print("Accuracy {}: {:.2f}".format(name, correct/total))

In [ ]:
validate(
        model = model, 
         train_loader = train_loader, 
         val_loader = val_loader
         )

In [ ]:
model2 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size= 3, padding= 1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 8, kernel_size= 3, padding= 1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(8, 16, kernel_size = 3, padding = 1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(256, 32),
            nn.Tanh(),
            nn.Linear(32,10)
)
model2.to('cuda:0')

In [ ]:
training_loop(n_epochs = 600, optimizer = optimizer, loss_fn = loss_fn, model = model2, train_loader = train_loader)

In [ ]:
validate(
        model = model2, 
         train_loader = train_loader, 
         val_loader = val_loader
         )

In [ ]:
import torch.nn.functional as F

In [ ]:
class ResBlock(nn.Module):
  def __init__(self, n_chans):
    super(ResBlock, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False)
    self.btach_norm = nn.BatchNorm2d(num_features=n_chans)
    torch.nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
    torch.nn.init.constant_(self.batch_norm.weight, 0.5)
    torch.nn.init.zeros_(self.batch_norm.bias)

  def forward(self, x):
    out = self.conv(x)
    out = torch.relu(out)
    return out + x
class ResNet10(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = ResNet10()
model.to('cuda:0')

In [ ]:
training_loop(n_epochs = n_epochs, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader)

In [ ]:
def validate(model, train_loader, val_loader):
  for name, loader in [("train", train_loader), ("val", val_loader)]:
    correct = 0
    total = 0
    with torch.no_grad():
      for imgs, labels in loader:
        outputs = model(imgs.to('cuda:0'))
        _, predicted = torch.max(outputs.to('cuda:0'), dim = 1)
        total += labels.shape[0]
        correct += int((predicted.to('cuda:0') == labels.to('cuda:0')).sum())
    print("Accuracy {}: {:.2f}".format(name, correct/total))

In [ ]:
validate(model = model, train_loader = train_loader, val_loader = val_loader)

In [ ]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn,
train_loader):
    device = 'cuda:0'
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                        for p in model.parameters())
            loss = loss + l2_lambda * l2_norm
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [ ]:
training_loop_l2reg(n_epochs = n_epochs, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader)

In [ ]:
validate(model = model, train_loader = train_loader, val_loader = val_loader)

In [ ]:
class ResBlock(nn.Module):
  def __init__(self, n_chans):
    super(ResBlock, self).__init__()
    self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False)
    self.dropout = nn.Dropout2d(p = 0.3)
  def forward(self, x):
    out = self.conv(x)
    out = self.dropout(out)
    out = torch.relu(out)
    return out + x

In [ ]:
class ResNet10D(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super().__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [ResBlock(n_chans=n_chans1)]))
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)
        self.fc2 = nn.Linear(32, 10)
    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2)
        out = out.view(-1, 8 * 8 * self.n_chans1)
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [259]:
model = ResNet10D()
model.to('cuda:0')
learning_rate = 3e-3
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()
loss_fn.to('cuda:0')
n_epochs = 300

In [260]:
training_loop(n_epochs = n_epochs, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader)

Epoch 1, Training Loss 0.005095358394905734
Epoch 1, Training Loss 0.009420937589367332
Epoch 1, Training Loss 0.012478083905661503
Epoch 1, Training Loss 0.015497911311781316
Epoch 1, Training Loss 0.018498081380448988
Epoch 1, Training Loss 0.021327377585193995
Epoch 1, Training Loss 0.024282243550585968
Epoch 1, Training Loss 0.027282910578696015
Epoch 1, Training Loss 0.030177978908314425
Epoch 1, Training Loss 0.03310792037593129
Epoch 1, Training Loss 0.03613773117894712
Epoch 1, Training Loss 0.039028957676704584
Epoch 1, Training Loss 0.04187473891031407
Epoch 1, Training Loss 0.04479264847152983
Epoch 1, Training Loss 0.04762487948093268
Epoch 1, Training Loss 0.05051382819709875
Epoch 1, Training Loss 0.05338879924296113
Epoch 1, Training Loss 0.056274268938147506
Epoch 1, Training Loss 0.05912391029660354
Epoch 1, Training Loss 0.062033231301075964
Epoch 1, Training Loss 0.06492223520108197
Epoch 1, Training Loss 0.06777564918293673
Epoch 1, Training Loss 0.07080416179374051

In [261]:
validate(model = model, train_loader = train_loader, val_loader = val_loader)

Accuracy train: 0.98
Accuracy val: 0.65


In [ ]:
model = ResNet10()
model.to('cuda:0')

In [ ]:
training_loop(n_epochs = n_epochs, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader)

In [ ]:
validate(model = model, train_loader = train_loader, val_loader = val_loader)